In [1]:
from func_def import *
df = pd.read_csv('data_pre_processed.csv',index_col=[0]).reset_index(drop=True)
df.tail(3)
get_streak(fighter='Randy Couture',time='2003-03-11',data=df)

,date,fighter,opponent,result
6997,2002-09-27,Randy Couture,Ricco Rodriguez,L
7026,2002-03-22,Randy Couture,Josh Barnett,L
7042,2001-11-02,Randy Couture,Pedro Rizzo,W
7066,2001-05-04,Randy Couture,Pedro Rizzo,W
7089,2000-11-17,Randy Couture,Kevin Randleman,W
7169,1997-12-21,Randy Couture,Maurice Smit,W
7178,1997-10-17,Randy Couture,Vitor Belfort,W
7193,1997-05-30,Randy Couture,Steven Graha,W
7195,1997-05-30,Randy Couture,Tony Halm,W


In [2]:
# Extract features
FEATURES = [item[item.find('_')+1:] for item in df.columns if 'f_' in item]

FULL_FEATURE_NAME = [f'f_{item}' for item in FEATURES] + [f'o_{item}' for item in FEATURES]
print(f'Constructing Features: {FEATURES}')

drop_columns = []
for column in FULL_FEATURE_NAME:
    if type(df.loc[0,column]) == str:
        drop_columns.append(column)
        df[f'{column}_succ'] = pd.to_numeric([item.split('of')[0] for item in df[column]])
        df[f'{column}_att'] = pd.to_numeric([item.split('of')[1] for item in df[column]])

df = df[[column for column in df.columns if column not in drop_columns]]
df.head(3)

Constructing Features: ['head_str', 'body_str', 'leg_str', 'distance_str', 'clinch_str', 'ground_str', 'head_str_perc', 'body_str_perc', 'leg_str_perc', 'dist_str_perc', 'clinch_str_perc', 'ground_str_perc']


,url,event_url,date,result,fighter,opponent,title,method,round,time,format,f_head_str_perc,f_body_str_perc,f_leg_str_perc,f_dist_str_perc,f_clinch_str_perc,f_ground_str_perc,o_head_str_perc,o_body_str_perc,o_leg_str_perc,o_dist_str_perc,o_clinch_str_perc,o_ground_str_perc,f_head_str_succ,f_head_str_att,f_body_str_succ,f_body_str_att,f_leg_str_succ,f_leg_str_att,f_distance_str_succ,f_distance_str_att,f_clinch_str_succ,f_clinch_str_att,f_ground_str_succ,f_ground_str_att,o_head_str_succ,o_head_str_att,o_body_str_succ,o_body_str_att,o_leg_str_succ,o_leg_str_att,o_distance_str_succ,o_distance_str_att,o_clinch_str_succ,o_clinch_str_att,o_ground_str_succ,o_ground_str_att
0,http://www.ufcstats.com/fight-details/1aa4c2fa...,http://www.ufcstats.com/event-details/c8a49ff2...,2023-10-07,L,Grant Dawson,Bobby Green,Lightweight Bout,KO/TKO,1,0:33,5 Rnd (5-5-5-5-5),0.33,0.34,0.33,0.33,0.34,0.33,1.00,0.00,0.00,0.14,0.00,0.85,0,6,0,0,0,0,0,6,0,0,0,0,7,10,0,0,0,1,1,4,0,0,6,7
1,http://www.ufcstats.com/fight-details/e966de63...,http://www.ufcstats.com/event-details/c8a49ff2...,2023-10-07,W,Joe Pyfer,Abdul Razak Alhassa,Middleweight Bout,Submission,2,2:05,3 Rnd (5-5-5),0.57,0.34,0.07,0.92,0.07,0.00,0.36,0.18,0.45,1.00,0.00,0.00,15,47,9,12,2,3,24,60,2,2,0,0,4,15,2,3,5,10,11,27,0,1,0,0
2,http://www.ufcstats.com/fight-details/eb76a5b7...,http://www.ufcstats.com/event-details/c8a49ff2...,2023-10-07,L,Alex Morono,Joaquin Buckley,Welterweight Bout,Decision - Unanimous,3,5:00,3 Rnd (5-5-5),0.72,0.25,0.02,0.92,0.07,0.00,0.60,0.35,0.04,0.80,0.02,0.16,29,122,10,17,1,2,37,133,3,8,0,0,41,134,24,36,3,6,55,141,2,9,11,26


In [3]:
# data enrichments - FIX WIN_STREAK
NEW_DATA_COLUMNS = ['win_perc','current_win_streak']
for column in NEW_DATA_COLUMNS:
    if column == 'win_perc':
        df[f'f_{column}'] = [get_win_perc(fighter=df.loc[i,'fighter'],time=df.loc[i,'date'],data=df) for i in tqdm(range(len(df)))]
        df[f'o_{column}'] = [get_win_perc(fighter=df.loc[i,'opponent'],time=df.loc[i,'date'],data=df) for i in tqdm(range(len(df)))]
    if column == 'current_win_streak':    
        df[f'f_{column}'] = [get_win_streak(fighter=df.loc[i,'fighter'],time=df.loc[i,'date'],data=df) for i in tqdm(range(len(df)))]
        df[f'o_{column}'] = [get_win_streak(fighter=df.loc[i,'opponent'],time=df.loc[i,'date'],data=df) for i in tqdm(range(len(df)))]

df.head(3)       

100%|██████████| 7239/7239 [00:28<00:00, 253.19it/s]


,url,event_url,date,result,fighter,opponent,title,method,round,time,format,f_head_str_perc,f_body_str_perc,f_leg_str_perc,f_dist_str_perc,f_clinch_str_perc,f_ground_str_perc,o_head_str_perc,o_body_str_perc,o_leg_str_perc,o_dist_str_perc,o_clinch_str_perc,o_ground_str_perc,f_head_str_succ,f_head_str_att,f_body_str_succ,f_body_str_att,f_leg_str_succ,f_leg_str_att,f_distance_str_succ,f_distance_str_att,f_clinch_str_succ,f_clinch_str_att,f_ground_str_succ,f_ground_str_att,o_head_str_succ,o_head_str_att,o_body_str_succ,o_body_str_att,o_leg_str_succ,o_leg_str_att,o_distance_str_succ,o_distance_str_att,o_clinch_str_succ,o_clinch_str_att,o_ground_str_succ,o_ground_str_att,f_win_perc,o_win_perc,f_current_win_streak,o_current_win_streak
0,http://www.ufcstats.com/fight-details/1aa4c2fa...,http://www.ufcstats.com/event-details/c8a49ff2...,2023-10-07,L,Grant Dawson,Bobby Green,Lightweight Bout,KO/TKO,1,0:33,5 Rnd (5-5-5-5-5),0.33,0.34,0.33,0.33,0.34,0.33,1.00,0.00,0.00,0.14,0.00,0.85,0,6,0,0,0,0,0,6,0,0,0,0,7,10,0,0,0,1,1,4,0,0,6,7,0.375000,0.181818,0,0
1,http://www.ufcstats.com/fight-details/e966de63...,http://www.ufcstats.com/event-details/c8a49ff2...,2023-10-07,W,Joe Pyfer,Abdul Razak Alhassa,Middleweight Bout,Submission,2,2:05,3 Rnd (5-5-5),0.57,0.34,0.07,0.92,0.07,0.00,0.36,0.18,0.45,1.00,0.00,0.00,15,47,9,12,2,3,24,60,2,2,0,0,4,15,2,3,5,10,11,27,0,1,0,0,0.500000,0.090909,0,0
2,http://www.ufcstats.com/fight-details/eb76a5b7...,http://www.ufcstats.com/event-details/c8a49ff2...,2023-10-07,L,Alex Morono,Joaquin Buckley,Welterweight Bout,Decision - Unanimous,3,5:00,3 Rnd (5-5-5),0.72,0.25,0.02,0.92,0.07,0.00,0.60,0.35,0.04,0.80,0.02,0.16,29,122,10,17,1,2,37,133,3,8,0,0,41,134,24,36,3,6,55,141,2,9,11,26,0.277778,0.400000,0,0


In [4]:
df.to_csv('data_processed.csv',index=None)
print('data_processed.csv saved.')
df.info()

data_processed.csv saved.
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7239 entries, 0 to 7238
Data columns (total 51 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   url                   7239 non-null   object 
 1   event_url             7239 non-null   object 
 2   date                  7239 non-null   object 
 3   result                7239 non-null   object 
 4   fighter               7239 non-null   object 
 5   opponent              7239 non-null   object 
 6   title                 7239 non-null   object 
 7   method                7239 non-null   object 
 8   round                 7239 non-null   int64  
 9   time                  7239 non-null   object 
 10  format                7239 non-null   object 
 11  f_head_str_perc       7239 non-null   float64
 12  f_body_str_perc       7239 non-null   float64
 13  f_leg_str_perc        7239 non-null   float64
 14  f_dist_str_perc       7239 non-null   float64
